<a href="https://colab.research.google.com/github/tvisha03/ISS-lab-work/blob/main/02_elliptic_curve_factorization_ecm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def gcd(a, b):
    # Euclidean algorithm to find GCD
    while b:
        a, b = b, a % b
    return a

def modinv(a, n):
    # Extended Euclidean Algorithm to compute modular inverse of a mod n
    t, new_t = 0, 1
    r, new_r = n, a
    while new_r != 0:
        quotient = r // new_r
        t, new_t = new_t, t - quotient * new_t
        r, new_r = new_r, r - quotient * new_r
    if r > 1:
        return None  # No inverse exists
    return t % n

In [2]:
def ec_add(P, Q, a, n):
    if P == 'INF': return Q
    if Q == 'INF': return P

    x1, y1 = P
    x2, y2 = Q

    if x1 == x2 and y1 != y2:
        return 'INF'

    if P == Q:
        numerator = (3 * x1 * x1 + a) % n
        denominator = (2 * y1) % n
    else:
        numerator = (y2 - y1) % n
        denominator = (x2 - x1) % n

    inv = modinv(denominator, n)
    if inv is None:
        factor = gcd(denominator, n)
        raise ValueError(f"Factor found! Non-invertible denominator: gcd({denominator}, {n}) = {factor}")

    slope = (numerator * inv) % n
    x3 = (slope * slope - x1 - x2) % n
    y3 = (slope * (x1 - x3) - y1) % n
    return (x3, y3)

In [3]:
def ec_scalar_mult(k, P, a, n):
    R = 'INF'
    while k > 0:
        if k % 2 == 1:
            R = ec_add(R, P, a, n)
        P = ec_add(P, P, a, n)
        k = k // 2
    return R

In [4]:
import random

def elliptic_curve_factorization(n):
    print(f"Trying to factor: {n}")

    while True:
        x = random.randint(1, n - 1)
        y = random.randint(1, n - 1)
        a = random.randint(1, n - 1)
        b = (y*y - x*x*x - a*x) % n

        print(f"Using curve: y^2 = x^3 + {a}x + {b} mod {n}")
        print(f"Starting point: P = ({x}, {y})")

        try:
            P = (x, y)
            for k in range(2, 100):  # Try increasing scalar multipliers
                P = ec_scalar_mult(k, P, a, n)
        except ValueError as e:
            print(str(e))
            break


In [5]:
# Example: Try factoring 8051 = 97 * 83
n = 10403

elliptic_curve_factorization(n)

Trying to factor: 10403
Using curve: y^2 = x^3 + 8213x + 2043 mod 10403
Starting point: P = (9816, 1410)
Factor found! Non-invertible denominator: gcd(7777, 10403) = 101
